In [1]:
%load_ext autoreload
%autoreload 2
import syft as sy
import numpy as np
import pandas as pd
from syft.core.adp.data_subject import DataSubject
from syft.core.tensor.smpc.mpc_tensor import MPCTensor
import time
import uuid
from syft.core.smpc.protocol.aby3 import ABY3
import operator
sy.logger.remove()

/home/azureuser/.local/share/virtualenvs/python39-2vM0bQN-/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DOMAIN1_PORT = 8081
DOMAIN2_PORT = 8082
p1 =  sy.login(email="info@openmined.org",password="changethis",port=DOMAIN1_PORT)
p2 = sy.login(email="info@openmined.org",password="changethis",port=DOMAIN2_PORT)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into canada... done!

Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into italy... done!


In [12]:
ROOT_DIR = "/home/azureuser/PySyft"
def load_data(csv_file: str) -> pd.DataFrame:
    return pd.read_csv(f"../trade_demo/datasets/{csv_file}")[0:10]

def get_user_details(unique_email: str) :
    return {
        "name": "Sheldon Cooper",
        "email": unique_email,
        "password": "bazinga",
        "budget": 10,
    }

unique_email = f"{uuid.uuid4()}@caltech.edu"

# Canada
ca_root = sy.login(email="info@openmined.org", password="changethis", port=DOMAIN1_PORT)
ca_data = load_data(csv_file="ca - feb 2021.csv")


# canada_trade = (
#     (np.array(list(ca_data["Trade Value (US$)"])) / 1000)[0:10]
# ).astype(np.int64)
canada_trade = np.array([[0,0],[0,0]])
trade_partners = ((list(ca_data["Partner"])))[0:10]

entities = list()
for i in range(len(trade_partners)):
    entities.append(DataSubject(name=trade_partners[i]))

sampled_canada_dataset = sy.Tensor(canada_trade)
sampled_canada_dataset.public_shape = sampled_canada_dataset.shape
sampled_canada_dataset = sampled_canada_dataset.private(
      0, 3, data_subjects=[entities[0].name]*canada_trade.shape[0]
).tag("trade_flow")

# load dataset
ca_root.load_dataset(
    assets={"Canada Trade": sampled_canada_dataset},
    name="Canada Trade Data - First few rows",
    description=(
        "A collection of reports from Canada's statistics bureau about how "
        + "much it thinks it imports and exports from other countries."
    ),
    skip_checks=True,
    
)

assert len(ca_root.datasets) > 0



# Italy
it_root = sy.login(email="info@openmined.org", password="changethis", port=DOMAIN2_PORT)
it_data = load_data(csv_file="it - feb 2021.csv")

# italy_trade = ((np.array(list(it_data["Trade Value (US$)"])) / 1000)[0:10]).astype(
#     np.int64
# )
italy_trade = np.array([[0,0],[0,0]])
trade_partners = ((list(it_data["Partner"])))[0:10]

entities = list()
for i in range(len(trade_partners)):
    entities.append(DataSubject(name="Other Asia, nes"))

# Upload a private dataset to the Domain object, as the root owner
sampled_italy_dataset = sy.Tensor(italy_trade)
sampled_italy_dataset.public_shape = sampled_italy_dataset.shape
sampled_italy_dataset = sampled_italy_dataset.private(
    0, 3, data_subjects=[entities[0].name]*canada_trade.shape[0]
).tag("trade_flow")

it_root.load_dataset(
    assets={"Italy Trade": sampled_italy_dataset},
    name="Italy Trade Data - First few rows",
    description=(
        "A collection of reports from iStat's statistics bureau about how "
        + "much it thinks it imports and exports from other countries."
    ),
    skip_checks=True,
    
)




Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into canada... done!


/tmp/ipykernel_173140/99810746.py:3: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f"../trade_demo/datasets/{csv_file}")[0:10]


Dataset is uploaded successfully !!! 🎉

Run `<your client variable>.datasets` to see your new dataset loaded into your machine!

Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into italy... done!
Dataset is uploaded successfully !!! 🎉

Run `<your client variable>.datasets` to see your new dataset loaded into your machine!


/tmp/ipykernel_173140/99810746.py:3: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f"../trade_demo/datasets/{csv_file}")[0:10]


In [13]:
from syft.core.node.common.node_service.user_manager.user_messages import (
    UpdateUserMessage,
)

# Upgrade admins budget
content = {"user_id": 1, "budget": 9_999_999}
p1._perform_grid_request(grid_msg=UpdateUserMessage, content=content)
p2._perform_grid_request(grid_msg=UpdateUserMessage, content=content)

print(p1.privacy_budget)
print(p2.privacy_budget)

9999999.0
9999999.0


In [14]:
x = p1.datasets[-1]["Canada Trade"]
y = p2.datasets[-1]["Italy Trade"]

In [15]:
# working operations
ops = ["add","sub","mul","matmul","lt","gt","eq","ne","ge","le"]
idx = 0 # points to the op
op = getattr(operator,ops[idx])
res = op(x,y)
#For comparison increase the timeout
res.block_with_timeout(10)
val = res.get_copy()

In [ ]:
val == op(sampled_canada_dataset.decode(),sampled_italy_dataset.decode())

In [16]:
print(val) #FPT result
print(op(sampled_canada_dataset.decode(),sampled_italy_dataset.decode())) #Acutal result

[[0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]]


In [ ]:
# DEBUG CELL
# print((sampled_canada_dataset).decode())
# print((sampled_italy_dataset).decode())
# print(op(sampled_canada_dataset.decode(),sampled_italy_dataset.decode()).round())
# print(res.get_copy().round())

In [17]:
p = res
public_result = p.publish(sigma=1)

In [ ]:
pb_res = public_result.get_copy()

In [ ]:
print(pb_res)
print(p.get_copy())

In [19]:
x = res * 10

In [7]:
t1 = res>0

Computing...:   0%|          | 0/64 [00:00<?, ?it/s]/home/azureuser/PySyft/packages/syft/src/syft/core/tensor/smpc/utils.py:72: RuntimeWarning: overflow encountered in multiply
  res = op(np.empty(x_shape), np.empty(y_shape)).shape
Computing...: 100%|██████████| 64/64 [00:54<00:00,  1.17it/s]


In [ ]:
t2 = res*t1

In [ ]:
print(t1.get_copy())
print(res.get_copy())
# t1.get_copy()*res.get_copy()

In [ ]:
res.get_copy()

In [ ]:
#Below are debug cells
a=np.array([ -789925275 ,  798267317, -1708906764,  1806674886,  1165022425,   480477472,
    54663083  ,  78462192,  -974424744, -1538872148],dtype=np.int32)
b=np.array([  789925275,  -798266317 , 1708920764, -1806674886, -1165022425,  -480477472,
   -54663083  , -78459192  , 974424744 , 1538872148],dtype=np.int32)

In [ ]:
(a+b)/1000

In [ ]:
sampled_canada_dataset.decode()

In [ ]:
a=np.array([-1362805391  ,1471796679 , 2126726498   ,874113086 , -532526475, -1815214451,
  1602118491,  1561978677 , -970006488 ,  792178136],dtype=np.int32)

b=np.array([ 1362805391, -1470796679 ,-2084726498  ,-874113086  , 532526475  ,1815214451,
 -1602118491 ,-1558978677 ,  970006488  ,-792178136],dtype=np.int32)


In [ ]:
a= np.array([ 1300758168, -1917863768, -1667359440,  2060242384 ,  -49469704 ,-1556715208,
   -95689592 ,   94451448 , -656120896, -1904153536],dtype=np.int32)

b=np.array( [-1300758168 ,-1377103528  , 717686480, -2060242384,    49469704,  1556715208,
    95689592, -1389418744 ,  656120896  ,1904153536],dtype=np.int32)

In [ ]:
res.child[0].get_copy()+res.child[1].get_copy()

In [ ]:
def value(a):
    v1 = a.child[0].get_copy()
    v2 = a.child[1].get_copy()
    v = v1+v2
    return v.child

In [8]:
v1 = t1.child[0].get_copy()
v2 = t1.child[1].get_copy()

In [11]:
v1.child.child.child + v2.child.child.child

ShareTensor(child=[[ True  True]
 [ True  True]])